In [1]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle

In [2]:
!pip uninstall kaggle

Uninstalling kaggle-1.5.9:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.9.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.9


In [3]:
!pip install kaggle

     |████████████████████████████████| 61kB 5.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-cp36-none-any.whl size=73265 sha256=9422f809b094a0a931e10b60078c702667bcf6a4a1cdffaa0b0856f36f51c484
  Stored in directory: /root/.cache/pip/wheels/68/6d/9b/7a98271454edcba3b56328cbc78c037286e787d004c8afee71
Successfully built kaggle


In [4]:

import kaggle

In [5]:
!kaggle competitions download -c dog-breed-identification

 98% 677M/691M [00:08<00:00, 32.5MB/s]
100% 691M/691M [00:08<00:00, 86.4MB/s]


In [6]:
!unzip -q dog-breed-identification.zip

In [12]:
import pandas as pd

In [13]:
labels = pd.read_csv("labels.csv")

In [14]:
labels.id = labels.id + ".jpg"

In [15]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [16]:
from tensorflow.keras.applications import vgg16

In [17]:
vgg_model = vgg16.VGG16(False,input_shape=(224,224,3))

58892288/58889256 [==============================] - 0s 0us/step


In [18]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [24]:
for layer in vgg_model.layers:
  layer.trainable = False
  print(layer.name,layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
idg = ImageDataGenerator(rescale=1/255,validation_split=0.1,rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,brightness_range=[0.2,0.4],zoom_range=0.4,horizontal_flip=True)

In [27]:
train_generator = idg.flow_from_dataframe(labels,"train",x_col="id",y_col="breed",target_size=(224,224),batch_size=64,subset="training")
val_generator = idg.flow_from_dataframe(labels,"train",x_col="id",y_col="breed",target_size=(224,224),batch_size=64,subset="validation")

Found 9200 validated image filenames belonging to 120 classes.
Found 1022 validated image filenames belonging to 120 classes.


In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD

In [ ]:
#we can add batch normalization between the convolutional layers 
#but we do not want to mess with weights of vgg16 model so we are adding Batch normalization layer between Dense layers

In [29]:
vgg_out = vgg_model.output
flat_out = Flatten()(vgg_out)
d1_out = Dense(1024,activation=relu)(flat_out)
bn1_out = BatchNormalization()(d1_out)
dp1_out = Dropout(0.25)(bn1_out)
d2_out=Dense(512,activation=relu)(dp1_out)
bn2_out= BatchNormalization()(d2_out)
dp2_out = Dropout(0.25)(bn2_out)
d3_out = Dense(120,activation=softmax)(dp2_out)
final_model = Model(inputs = [vgg_model.input],outputs=[d3_out])
final_model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [30]:
for layer in final_model.layers:
  print(layer.name,layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten_1 True
dense_1 True
batch_normalization_1 True
dropout True
dense_2 True
batch_normalization_2 True
dropout_1 True
dense_3 True


In [31]:
final_model.compile(optimizer=SGD(),loss=categorical_crossentropy,metrics=["acc"])

In [32]:
final_model.fit_generator(train_generator,epochs=10,validation_data=val_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
144/144 [==============================] - 169s 1s/step - loss: 5.4280 - acc: 0.0147 - val_loss: 5.2867 - val_acc: 0.0068
Epoch 2/10
144/144 [==============================] - 169s 1s/step - loss: 5.1027 - acc: 0.0236 - val_loss: 4.8966 - val_acc: 0.0284
Epoch 3/10
144/144 [==============================] - 169s 1s/step - loss: 4.9192 - acc: 0.0322 - val_loss: 4.9429 - val_acc: 0.0362
Epoch 4/10
144/144 [==============================] - 163s 1s/step - loss: 4.8257 - acc: 0.0361 - val_loss: 4.7152 - val_acc: 0.0333
Epoch 5/10
144/144 [==============================] - 164s 1s/step - loss: 4.7069 - acc: 0.0454 - val_loss: 4.6912 - val_acc: 0.0411
Epoch 6/10
144/144 [==============================] - 165s 1s/step - loss: 4.6491 - acc: 0.0486 - val_loss: 4.5108 - val_acc: 0.0607
Epoch 7/10
144/144 [==============================] - 165s 1s/step - loss: 4.6034 - acc: 0.0560 - val_loss: 4.5083 - val_acc: 

In [ ]:
#saturation will be met at later epochs like 15 or 20
#all these are techniques to overcome overfitting in model training so our model is generalized

In [ ]:
+